In [1]:
import pandas as pd
import numpy as np

In [2]:
# vviews_files = !ls ../../data | grep vviews
starts_file = 'skillshare_2022_starts.csv'
classes_file = 'skillshare_2022_classes.csv'
vviews_files = [f"skillshare_2022_vviews_{i}.csv" 
                for i in range(0,3)]
                #for i in range(0,63)]

In [3]:
def load_csv_to_df(filepath):
    return pd.read_csv("../../data/" + filepath, index_col=0)

vviews = pd.DataFrame()
for file in vviews_files:
    vviews = pd.concat([vviews, load_csv_to_df(file)])
    
starts = load_csv_to_df(starts_file)
for column in starts.columns:
    if "time" in column:
        starts[column] = pd.to_datetime(starts[column])
        
classes = load_csv_to_df(classes_file)
    
vviews.view_date = pd.to_datetime(vviews.view_date)
vviews.head()

/tmp/ipykernel_44699/3826698520.py:2: DtypeWarning: Columns (24,53) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv("../../data/" + filepath, index_col=0)
/tmp/ipykernel_44699/3826698520.py:2: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv("../../data/" + filepath, index_col=0)


,uid,class_id,lesson_id,lesson_number,is_trailer,video_duration,view_date,sum
0,1128075,121998,1533011,6,False,407.0,2022-01-07,333
1,23721429,285819,2992845,3,False,379.0,2022-01-07,215
2,23747223,151243,2482675,5,False,372.0,2022-01-07,192
3,23630537,108140,652188,19,False,203.0,2022-01-07,24
4,23693292,516912,4244299,3,False,340.0,2022-01-07,89


In [4]:
classes.head()

,class_id,class_title,create_time,description,takeaway,category,subcategory,status,is_featured,is_premium_only,...,language_grade_notes_all,has_non_english_content,non_english_language_1,class_origin,class_origin_raw,published_class_number,teacher_first_publish_time,is_last_published_class,is_first_original_class,is_first_quality_class
0,23577,A Crash Course in Photographic Composition,2013-01-22 17:41:39,<p>This class is for anybody who wants to crea...,NaN,Creative,Photography,published,True,True,...,NaN,False,NaN,Other,No Source,1,2013-01-22 17:41:39,True,False,True
1,51370,Drawing Time: Illustrator Techniques,2013-01-31 23:54:50,"<p><img src=""https://static.skillshare.com/upl...",NaN,Creative,Illustration,published,True,True,...,NaN,False,NaN,Other,No Source,1,2013-01-31 23:54:50,True,False,False
2,55284,Perfect Southern Fried Chicken + Buttermilk Bi...,2013-02-01 23:50:57,<p><strong>Make The Best Southern Fried Chicke...,NaN,Lifestyle,Culinary,published,True,True,...,NaN,False,NaN,Other,No Source,1,2013-02-01 23:50:57,True,False,False
3,65056,EasyDSLR: Master your Digital Camera in less t...,2013-02-05 00:16:27,<p>So you finally got that DSLR camera you've ...,NaN,NaN,Photography,published,True,True,...,NaN,False,NaN,Other,No Source,1,2013-02-05 00:16:27,True,False,False
4,100968,Pattern Design: Creating Repeat Patterns From ...,2013-03-30 04:02:06,<p>Ever wanted to create your own hand-drawn p...,NaN,Creative,Graphic Design,published,True,True,...,NaN,False,NaN,Other,No Source,1,2013-03-30 04:02:06,True,False,False


In [5]:
classes.subcategory.unique()

array(['Photography', 'Illustration', 'Culinary', 'Graphic Design',
       'Marketing', 'Creative Writing', 'Music', 'UI/UX Design', 'Other',
       'Data Science', nan, 'Crafts', 'Animation',
       'Freelance & Entrepreneurship', 'Game Design', 'Film & Video',
       'IT Security', 'Leadership & Management', 'Web Development',
       'Finance', 'Sales', 'Human Resources', 'Fine Art', 'Gaming',
       'Productivity', 'Languages', 'Business Analytics', 'Home Business',
       'Product Management', 'Teaching', 'Mobile Development',
       'Health & Wellness', 'Accounting'], dtype=object)

In [6]:
# tack on class info to video views
vviews = pd.merge(
    vviews,
    classes[["class_id", "category", "subcategory", "has_non_english_content"]],
    on="class_id",
    how="left"
)
vviews.head()

,uid,class_id,lesson_id,lesson_number,is_trailer,video_duration,view_date,sum,category,subcategory,has_non_english_content
0,1128075,121998,1533011,6,False,407.0,2022-01-07,333,Lifestyle,Crafts,False
1,23721429,285819,2992845,3,False,379.0,2022-01-07,215,Creative,Animation,False
2,23747223,151243,2482675,5,False,372.0,2022-01-07,192,Business,Productivity,False
3,23630537,108140,652188,19,False,203.0,2022-01-07,24,Lifestyle,Languages,False
4,23693292,516912,4244299,3,False,340.0,2022-01-07,89,Lifestyle,Teaching,False


In [7]:
watchtime_by_category = vviews.groupby([
    "uid",
    "subcategory"])\
    .sum()["sum"]\
    .reset_index()\
    .pivot(
        index="uid",
        columns="subcategory",
        values="sum")\
    .fillna(0.0)

watchtime_by_category["Total"] = watchtime_by_category.sum(axis=1)

# Convert to percentage
for col in watchtime_by_category.columns[:-2]:
    watchtime_by_category[col] = watchtime_by_category[col] / \
        watchtime_by_category["Total"]
watchtime_by_category

subcategory,Accounting,Animation,Business Analytics,Crafts,Creative Writing,Culinary,Data Science,Film & Video,Finance,Fine Art,...,Music,Other,Photography,Product Management,Productivity,Sales,Teaching,UI/UX Design,Web Development,Total
uid,,,,,,,,,,,,,,,,,,,,,
526,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,3471.0
7329,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,4109.0
21246,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,4232.0
69897,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,8022.0
82347,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25473351,0.000000,0.686573,0.000000,0.0,0.0,0.0,0.0,0.000479,0.0,0.000000,...,0.0,0.0,0.082743,0.0,0.032740,0.0,0.0,0.00000,0.0,117013.0
25686733,0.696716,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.220825,0.0,0.0,0.08246,0.0,1431.0
25784827,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,3137.0
